# Tarea Grupal Redes Neuronales

### Función de carga de datos desde el directorio FER

In [18]:
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, add
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import numpy as np
import math
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image


def extraer_imagenes(directorio):
    imagenes_train, clases_train = [], []
    imagenes_test, clases_test = [], []

    for tipo_datos in ['train', 'test']:
        ruta_tipo = os.path.join(directorio, tipo_datos)
        clases = sorted(os.listdir(ruta_tipo))  # Asegura orden consistente de clases

        for idx, clase in enumerate(clases):
            ruta_clase = os.path.join(ruta_tipo, clase)
            for foto in os.listdir(ruta_clase):
                ruta_foto = os.path.join(ruta_clase, foto)
                try:
                    img = image.load_img(ruta_foto, target_size=(48, 48), color_mode='grayscale')
                    img_array = np.array(img)

                    if tipo_datos == 'train':
                        imagenes_train.append(img_array)
                        clases_train.append(idx)
                    else:
                        imagenes_test.append(img_array)
                        clases_test.append(idx)
                except Exception as e:
                    print(f"Error al cargar {ruta_foto}: {e}")

    # Convertir a arrays y normalizar
    X_train = np.expand_dims(np.array(imagenes_train, dtype=np.float32) / 255.0, axis=-1)
    X_test = np.expand_dims(np.array(imagenes_test, dtype=np.float32) / 255.0, axis=-1)
    y_train = to_categorical(clases_train)
    y_test = to_categorical(clases_test)

    return X_train, X_test, y_train, y_test

### Definicón de parámertros y arquitectura, además de cargar los datos del directorio
-   Para cumplir con lo pedido en el punto a de la pregunta 1, se diseñaron 3 archivos.py, que ejecutarán el modelo con los factores de compresión solicitados (0.3, 0.5, 0.7), Los archivos se llaman de la siguiente manera:

    - Punto_a_1.py = factor compresión 0.3
    - Punto_a_2.py = factor compresión 0.5
    - Punto_a_3.py = factor compresión 0.7

In [8]:
# training parameters
batch_size = 32
#epochs = 200
epochs = 200
data_augmentation = True

# network parameters
num_classes = 7
num_dense_blocks = 4
use_max_pool = False

growth_rate = 12
depth = 100
num_bottleneck_layers = (depth - 4) // (2 * num_dense_blocks)

num_filters_bef_dense_block = 2 * growth_rate
compression_factor = 0.5

# load the CIFAR10 data
# Cargar los datos de CIFAR10.
x_train, x_test, y_train, y_test = extraer_imagenes("FER")
# input image dimensions
input_shape = x_train.shape[1:]

# mormalize data
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices.


def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

x_train shape: (28709, 48, 48, 1)
y_train shape: (28709, 7)
28709 train samples
7178 test samples


### Se diseña el modelo DenseNet 

In [9]:
# start model definition
# densenet CNNs (composite function) are made of BN-ReLU-Conv2D
inputs = Input(shape=input_shape)
x = BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(num_filters_bef_dense_block,kernel_size=3,padding='same',kernel_initializer='he_normal')(x)
x = concatenate([inputs, x])

# stack of dense blocks bridged by transition layers
for i in range(num_dense_blocks):
    # a dense block is a stack of bottleneck layers
    for j in range(num_bottleneck_layers):
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(4 * growth_rate,kernel_size=1,padding='same',kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(growth_rate,kernel_size=3,padding='same',kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        x = concatenate([x, y])

    # no transition layer after the last dense block
    if i == num_dense_blocks - 1:
        continue

    # transition layer compresses num of feature maps and reduces the size by 2
    num_filters_bef_dense_block += num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block * compression_factor)
    y = BatchNormalization()(x)
    y = Conv2D(num_filters_bef_dense_block,kernel_size=1,padding='same',kernel_initializer='he_normal')(y)
    if not data_augmentation:
        y = Dropout(0.2)(y)
    x = AveragePooling2D()(y)


# add classifier on top
# after average pooling, size of feature map is 1 x 1
x = AveragePooling2D(pool_size=4)(x)
y = Flatten()(x)
outputs = Dense(num_classes,kernel_initializer='he_normal',activation='softmax')(y)

# instantiate and compile model
# orig paper uses SGD but RMSprop works better for DenseNet
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(1e-3),metrics=['acc'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 batch_normalization_200 (Batch  (None, 48, 48, 1)   4           ['input_3[0][0]']                
 Normalization)                                                                                   
                                                                                                  
 activation_194 (Activation)    (None, 48, 48, 1)    0           ['batch_normalization_200[0][0]']
                                                                                                  
 conv2d_200 (Conv2D)            (None, 48, 48, 24)   240         ['activation_194[0][0]']   

### Modelos ResNet utilizados

In [19]:
from tensorflow.keras.regularizers import l2

def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1,
                 activation='relu', batch_normalization=True, conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder"""
    conv = Conv2D(num_filters,
                  kernel_size =kernel_size,
                  strides     =strides,
                  padding     ='same',
                  kernel_initializer ='he_normal',
                  kernel_regularizer =l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x



## ResNet V1

In [25]:
batch_size = 64
epochs = 200
data_augmentation = True
n_clases = 7 #clases 
subtract_pixel_mean = True

n = 17  
depth = n * 6 + 2  # 104 capas

# Implementación ResNet V1
def resnet_v1(input_shape, depth, n_clases=7):
    """ResNet Version 1 Model"""
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (e.g. 20, 32, 44, 56, 110, 164)')
    
    # Empezar definición del modelo
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    
    # Instanciar la pila de unidades residuales
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # primera capa pero no primer stack
                strides = 2  # downsampling
            y = resnet_layer(inputs=x, num_filters=num_filters, strides=strides)
            y = resnet_layer(inputs=y, num_filters=num_filters, activation=None)
            
            if stack > 0 and res_block == 0:
                # proyección lineal de la conexión residual
                x = resnet_layer(inputs=x, num_filters=num_filters, kernel_size=1,
                                strides=strides, activation=None, batch_normalization=False)
            x = add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Añadir clasificador
    x = AveragePooling2D(pool_size=4)(x)  # Ajustado para tamaño 48x48
    y = Flatten()(x)
    outputs = Dense(n_clases, activation='softmax', kernel_initializer='he_normal')(y)

    # Instanciar modelo
    model = Model(inputs=inputs, outputs=outputs)
    return model
    # Definir la forma de la entrada (48x48x1 para imágenes en escala de grises)
input_shape = (48, 48, 1)
    
    # Crear modelo ResNet V2
model = resnet_v1(input_shape=input_shape, depth=depth, n_clases=n_clases)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 conv2d_157 (Conv2D)            (None, 48, 48, 16)   160         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_154 (Batch  (None, 48, 48, 16)  64          ['conv2d_157[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_154 (Activation)    (None, 48, 48, 16)   0           ['batch_normalization_154[0

## ResNet V2

In [23]:
batch_size = 64
epochs = 200
data_augmentation = True
n_clases = 7 #clases 
subtract_pixel_mean = True #Mejora accuracy

# Para ResNet V2, la profundidad se calcula diferente
n = 17
depth = n * 9 + 2  # 155 capas
# Implementación ResNet V2
def resnet_v2(input_shape, depth, n_clases=7):
    """ResNet Version 2 Model con pre-activación"""
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (e.g. 29, 47, 56, 92, 110, 164)')
    
    #definición del modelo
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    
    # v2 realiza Conv2D con BN-ReLU en la entrada antes de dividirse en caminos
    x = resnet_layer(inputs=inputs)
    
    for stack in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stack == 0:
                # Primer stage mantiene resolución, duplica filtros
                num_filters_out = num_filters_in * 4
                # Primera capa y primer stage
                if res_block == 0:  
                    activation = None
                    batch_normalization = False
            else:
                # Otros stages reducen resolución, duplican filtros
                num_filters_out = num_filters_in * 2
                # Primera capa pero no primer stage
                if res_block == 0:
                    # downsample
                    strides = 2
            
            # Bloque residual con pre-activación (BN-ReLU-Conv)
            y = resnet_layer(inputs=x,num_filters=num_filters_in,kernel_size=1,strides=strides,
                             activation=activation,batch_normalization=batch_normalization,conv_first=False)# Primera parte del bloque
            y = resnet_layer(inputs=y,num_filters=num_filters_in,conv_first=False)# Segunda parte 
            y = resnet_layer(inputs=y,num_filters=num_filters_out,kernel_size=1,conv_first=False)# Tercera parte 
            # Proyección de la conexión residual si cambia dimensiones
            if res_block == 0:
                # Proyección lineal para ajustar dimensiones
                x = resnet_layer(inputs=x,num_filters=num_filters_out,kernel_size=1,
                    strides=strides,activation=None,batch_normalization=False)
            
            x = add([x, y])
        
        num_filters_in = num_filters_out
    
    # En ResNet V2, BN-ReLU antes del Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Añadir clasificador
    x = AveragePooling2D(pool_size=4)(x)  # Ajustado para tamaño 48x48
    y = Flatten()(x)
    outputs = Dense(n_clases, activation='softmax', kernel_initializer='he_normal')(y)

    #Intalación del modelo
    model = Model(inputs=inputs, outputs=outputs)
    return model


    
    # Definir la forma de la entrada (48x48x1 para imágenes en escala de grises)
input_shape = (48, 48, 1)
    
    # Crear modelo ResNet V2
model = resnet_v2(input_shape=input_shape, depth=depth, n_clases=n_clases)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 48, 48, 16)   160         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 48, 48, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 48, 48, 16)   0           ['batch_normalization[0][0]']

### Se testean los mejores modelos obtenidos tras el entrenamiento en KHIPU

## DenseNet 0.3

In [17]:
from tensorflow.keras.models import load_model

# Cargar el modelo
model_03 = load_model('saved_models_200_epocas_0.3_compression_factor\FER2013_densenet_model_compression_factor_0.3.84.h5')

X_train, X_test, y_train, y_test = extraer_imagenes("FER")

# Evaluar
loss, accuracy = model_03.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

225/225 [==============================] - 32s 136ms/step - loss: 1.1375 - acc: 0.6828
Test accuracy: 0.6828


## DenseNet 0.5

In [6]:
# Cargar el modelo
model_05 = load_model('saved_models_200_epocas_0.5_compression_factor\FER2013_densenet_model_compression_factor_0.5.106.h5')

# Evaluar
loss, accuracy = model_05.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

225/225 [==============================] - 33s 140ms/step - loss: 1.8759 - acc: 0.6909
Test accuracy: 0.6909


## DenseNet 0.7

In [11]:
# Cargar el modelo
model_07 = load_model('saved_models_200_epocas_0.7_compression_factor\FER2013_densenet_model_compression_factor_0.7.92.h5')

# Evaluar
loss, accuracy = model_07.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

225/225 [==============================] - 35s 149ms/step - loss: 1.2770 - acc: 0.6934
Test accuracy: 0.6934


## ResNet V1

In [9]:
# Cargar el modelo
model_v1 = load_model("saved_models/fer2013_resnet104v1.h5")

# Evaluar
loss, accuracy = model_v1.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

225/225 [==============================] - 23s 93ms/step - loss: 1.9426 - accuracy: 0.6652
Test accuracy: 0.6652


## ResNet V2

In [10]:
# Cargar el modelo
model_v2 = load_model('saved_models/fer2013_resnet155v2.h5')

# Evaluar
loss, accuracy = model_v2.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")

225/225 [==============================] - 34s 142ms/step - loss: 1.4614 - accuracy: 0.6627
Test accuracy: 0.6627
